# Extract, Transform and Load

Load libraries

In [132]:
import pandas as pd
from sqlalchemy import create_engine

In [133]:
csv_file = "Labor Data\ID Roster 08_30_22.csv"

positions_df = pd.read_csv(csv_file)
positions_df.head()

,UIK,Shift,Title,Base,Incentive,Base OT,Incentive OT
0,2745579,3,Line Lead I,$19.00,$21.00,$28.50,$31.50
1,2745578,3,Line Lead I,$19.00,$21.00,$28.50,$31.50
2,2745577,1,PIT 1st shift,$17.99,$18.99,$26.99,$28.49
3,2745564,2,Line Lead I,$19.00,$21.00,$28.50,$31.50
4,2745551,1,PIT 2nd shift,$18.01,$19.01,$27.01,$28.51


## Table 1: Positions 

Attributes:
* Associate UIK
*Shift
*Position
*Base wage
*Incentive
*Base OT
*Incentive OT

In [134]:
# Look at the different names of positions and names with errors
positions = positions_df["Title"].unique()
positions

array(['Line Lead I', 'PIT 1st shift', 'PIT 2nd shift', 'SAP',
       'Quality Lead 3rd shift', 'Sanitation', 'Salaried',
       'Lead, Quality Control and Food Safety', 'Line Lead II',
       'Quality Control & Food Safety - 1st Shift',
       'Coordinator, Production (Outbound)', 'Trainer 1st shift',
       'Packers 1st shift', 'Packer 1st shift', 'Packer 2nd shift',
       'Packer 1st Shift', 'Maintenance ',
       'Quality Control & Food Safety 2nd Shift', 'Packer 3rd Shift',
       'Packers 2nd shift', 'Packer 2nd Shift', 'Security',
       'Packer 1st shidt', 'Donor coordinator', 'PIT driver 2nd shift',
       'PIT 3rd shift', 'PIT 1 shfit', 'Trainer 3rd shift',
       'Line lead II', 'Quality lead  2nd shift',
       'Quality Control & Food Safety - 2nd Shift',
       'Quality Control & Food Safety 3rd shift', 'Maintenance',
       'Quality Lead', 'Administrative Assistant'], dtype=object)

In [135]:
# Standardize position names
standard_position_names = {'Packer 2nd shift':"Material Handler", 
                           'Packer 2nd Shift':"Material Handler", 
                           'Packer 1st shift':"Material Handler",
                           'Packers 2nd shift':"Material Handler",
                           'Packers 1st shift':"Material Handler",
                           'Packer 1st shidt':"Material Handler",
                           'Packer 1st Shift':"Material Handler", 
                           'Packer 3rd Shift':"Material Handler",
                           'Quality Control & Food Safety - 1st Shift':"QA Tech",
                           'Quality Control & Food Safety 3rd shift':"QA Tech",
                           'Quality Control & Food Safety - 2nd Shift':"QA Tech",
                           'Quality Control & Food Safety 2nd Shift':"QA Tech", 
                           'Quality lead  2nd shift':"Quality and Food Safety Lead",
                           'Quality Lead 3rd shift':"Quality and Food Safety Lead", 
                           'Quality Lead':"Quality and Food Safety Lead",
                           'Lead, Quality Control and Food Safety':"Quality and Food Safety Lead",
                           'PIT 1st shift':"FLO",
                           'PIT 3rd shift':"FLO",
                           'PIT 2nd shift':"FLO",
                           'PIT 1 shfit':"FLO",
                           'PIT driver 2nd shift':"FLO",
                           'Trainer 3rd shift':"Machine Operator Trainer",
                           'Trainer 1st shift':"Machine Operator Trainer", 
                           'Security':"Secruity Guard",
                           'Maintenance':"Maintenance Technician",
                           'Maintenance ':"Maintenance Technician",
                           'Line Lead II':'Line Lead',
                           'Line Lead I':'Line Lead', 
                           'Line lead II':'Line Lead',
                           'Donor coordinator':"Lead Donor Coordinator",
                           'Administrative Assistant':"Office Administrator",'SAP':"SAP Administrator",
                           'Sanitation':"Site Sanitation", 
                           'Coordinator, Production (Outbound)':"Shift Donor Receiver" }

In [136]:
# Replace position names with standardized names
positions_df_clean = positions_df.replace({"Title": standard_position_names})
positions = positions_df_clean["Title"].unique()
positions

array(['Line Lead', 'FLO', 'SAP Administrator',
       'Quality and Food Safety Lead', 'Site Sanitation', 'Salaried',
       'QA Tech', 'Shift Donor Receiver', 'Machine Operator Trainer',
       'Material Handler', 'Maintenance Technician', 'Secruity Guard',
       'Lead Donor Coordinator', 'Office Administrator'], dtype=object)

In [137]:
# Load data with unique identifiers: UIK
csv_file = "Labor Data\Associate_UIK.csv"
associates_df = pd.read_csv(csv_file)
positions_df.rename(columns = {'UIK':"Associate UIK",'Title':'Position'}, inplace = True)
positions_df.head()

,Associate UIK,Shift,Position,Base,Incentive,Base OT,Incentive OT
0,2745579,3,Line Lead I,$19.00,$21.00,$28.50,$31.50
1,2745578,3,Line Lead I,$19.00,$21.00,$28.50,$31.50
2,2745577,1,PIT 1st shift,$17.99,$18.99,$26.99,$28.49
3,2745564,2,Line Lead I,$19.00,$21.00,$28.50,$31.50
4,2745551,1,PIT 2nd shift,$18.01,$19.01,$27.01,$28.51


## Table 2: Associates

Attributes:
* Associate UIK
*Position 

In [138]:
# Eliminate unnecessary columns
associates_df = positions_df[['Associate UIK', 'Position']]
associates_df.head()

,Associate UIK,Position
0,2745579,Line Lead I
1,2745578,Line Lead I
2,2745577,PIT 1st shift
3,2745564,Line Lead I
4,2745551,PIT 2nd shift


## Table 3: SKU Master

Attributes:
* SKU ID
*Run Rate
*Hourly Run Rate
*Case Cost


In [139]:
csv_file = "Labor Data\Price Model.csv"
SKU_Master_df = pd.read_csv(csv_file)

In [140]:
SKU_Master_df.head()

,SKU ID,2023 PTUs,PTU/HR,PTU/shift,# of people on Line,Base Direct Labor Rate ($/hr),QA Tech,Line Lead,Production FLO,Warehouse FLO,"Supervisors, Trainers, Donor Coordinator, Donor Receivers, FG Leads, Baler MO",Cleaning,Total Flex Cost,Total Unit Cost (Direct + Flex)
0,428207918,0,32.50,260.0,13.0,16.85,0.587963,1.462734,0.591233,0.633213,1.317110,0.0,4.592254,14.50
1,428484365,0,47.50,380.0,14.0,16.85,0.402291,1.000818,0.404528,0.433251,0.901181,0.0,3.142069,10.44
2,429261590,0,96.25,770.0,14.0,16.85,0.198533,0.493910,0.199637,0.213812,0.444739,0.0,1.550631,5.15
3,424231364,2025,90.00,720.0,10.0,16.85,0.212320,0.528209,0.213501,0.228660,0.475623,0.0,1.658314,4.41
4,426282894,162373,81.25,650.0,21.0,16.85,0.235185,0.585094,0.236493,0.253285,0.466220,0.0,1.776277,8.18


In [142]:
# Eliminate unnecessary columns
SKU_Master_df = SKU_Master_df[['SKU ID', 'PTU/shift', 'PTU/HR', 'Total Unit Cost (Direct + Flex)']]
SKU_Master_df.head()

,SKU ID,PTU/shift,PTU/HR,Total Unit Cost (Direct + Flex)
0,428207918,260.0,32.50,14.50
1,428484365,380.0,47.50,10.44
2,429261590,770.0,96.25,5.15
3,424231364,720.0,90.00,4.41
4,426282894,650.0,81.25,8.18


In [143]:
# Change column names as per the standard
SKU_Master_df.rename(columns = {'PTU/shift':"Run Rate",
                                'PTU/HR':'Hourly Run Rate', 
                                'Total Unit Cost (Direct + Flex)':'Case Cost'}, inplace = True)
SKU_Master_df.head()

,SKU ID,Run Rate,Hourly Run Rate,Case Cost
0,428207918,260.0,32.50,14.50
1,428484365,380.0,47.50,10.44
2,429261590,770.0,96.25,5.15
3,424231364,720.0,90.00,4.41
4,426282894,650.0,81.25,8.18
